In [1]:
import pandas as pd
import requests
import csv
from datetime import datetime

In [2]:
API_KEY = "use_your_api_key"
regions = ["US", "GB", "CA", "IN", "AU", "DE", "FR", "IT", "ES", "MX", "BR", "JP", "KR", "RU", 
           "AR", "CO", "CL", "PE", "VE", "ZA", "NG", "PH", "TH", "ID", "MY", "SG", "HK", "TW", 
           "PL", "NL", "BE", "AT", "CH", "SE", "NO", "DK", "FI", "IE", "HU", "CZ", "RO", "GR", 
           "TR", "SA", "AE", "KW", "QA", "OM", "JO", "BD", "LK", "MM", "KH", "LA", "NP", "TW", 
           "SG", "MD", "BY", "UA", "LT", "LV", "EE", "RS", "HR", "SI", "BG", "MK", "AL", "MT", 
           "IS", "GL", "SM", "AD", "MC", "VA", "LI", "AW", "BQ", "CW", "SX"]



In [3]:
def fetch_trending_videos(region_code, max_results=200):
    base_url = "https://www.googleapis.com/youtube/v3/videos"
    all_videos = []
    page_token = ""

    while len(all_videos) < max_results:
        params = {
            "part": "snippet,statistics",
            "chart": "mostPopular",
            "regionCode": region_code,
            "maxResults": 50,
            "pageToken": page_token,
            "key": API_KEY
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if "items" not in data:
            break

        for item in data["items"]:
            video_data = {
                "video_id": item["id"],
                "trending_date": datetime.now().strftime('%Y-%m-%d'),
                "title": item["snippet"]["title"],
                "channel_title": item["snippet"]["channelTitle"],
                "category_id": item["snippet"]["categoryId"],
                "publish_time": item["snippet"]["publishedAt"],
                "tags": ','.join(item["snippet"].get("tags", [])),
                "views": item["statistics"].get("viewCount", 0),
                "likes": item["statistics"].get("likeCount", 0),
                "dislikes": item["statistics"].get("dislikeCount", 0),
                "comment_count": item["statistics"].get("commentCount", 0),
                "thumbnail_link": item["snippet"]["thumbnails"]["high"]["url"],
                "comments_disabled": 'commentCount' not in item["statistics"],
                "ratings_disabled": 'likeCount' not in item["statistics"],
                "video_error_or_removed": item["snippet"].get("video_error_or_removed", False),
                "description": item["snippet"]["description"],
                "region": region_code
            }
            all_videos.append(video_data)

        # Update the page token for the next page (if exists)
        page_token = data.get("nextPageToken", "")
        if not page_token:
            break

    return pd.DataFrame(all_videos)

# Initialize an empty DataFrame to store all videos
all_videos_df = pd.DataFrame()

# Fetch and append data for each region
for region in regions:
    videos_df = fetch_trending_videos(region, max_results=200)
    all_videos_df = pd.concat([all_videos_df, videos_df], ignore_index=True)

# Save the DataFrame to a CSV file
all_videos_df.to_csv("trending_videos_new.csv", index=False)

# Print the total number of videos fetched
print(f"Total videos fetched: {len(all_videos_df)}")


Total videos fetched: 10466
